In [9]:
import pandas as pd

df = pd.read_csv('Experimentos.csv')

# Normaliza a coluna removendo o %
df["Uso Máx. CPU"] = df["Uso Máx. CPU"].astype(str).str.replace("%", "")

# Convertendo colunas numéricas (e substituir vírgula decimal por ponto)
numeric_columns = [
    "Nº Usuários", "Taxa Erros (%)", "Menor Duração", "Maior Duração", "Duração Média", "Conexões Ativas",
    "Tam. Arq. Temp. (GB)", "Cache Hit (%)", "Uso Máx. CPU", "Uso Máx. Memória (GB)"
]
for col in numeric_columns:
    df[col] = df[col].astype(str).str.replace(",", ".").astype(float)

## Primeira abordagem

Você deve mudar os pesos de acordo com o seu conhecimento na área

In [15]:
import pandas as pd

# Função de normalização
def normalize(series, invert=False):
    min_val = series.min()
    max_val = series.max()
    norm = (series - min_val) / (max_val - min_val)
    return 1 - norm if invert else norm

# Vamos definir os pesos para cada métrica (ajuste conforme a importância)
pesos = {
    "Nº Usuários": 2.0,                 # Quanto maior número de usuários, melhor
    "Taxa Erros (%)": -1.0,             # Quanto menor a taxa de erros, melhor
    "Menor Duração": -1.0,              # Quanto menor a duração média, melhor
    "Maior Duração": -1.0,              # Quanto menor a duração máxima é melhor
    "Duração Média": -2.0,              # Quanto menor a duração média é melhor
    "Conexões Ativas": -1.0,            # Quanto menos conexoes ativas é melhor
    "Tam. Arq. Temp. (GB)": -1.0,       # Quanto menos arquivo temporario é melhor
    "Cache Hit (%)": 1.0,               # Quanto maior uso da cache é melhor
    "Uso Máx. CPU": -1.0,               # Quanto menos uso de CPU é melhor
    "Uso Máx. Memória (GB)": -2.0       # Quanto menos uso de memória é melhor
}

# Normalizando e aplicando os pesos
df_norm = pd.DataFrame()

# Normalizar "Nº Usuários" (quanto maior, melhor)
df_norm["Nº Usuários"] = normalize(df["Nº Usuários"]) * pesos["Nº Usuários"]

# Normalizar "Taxa Erros (%)" (quanto menor, melhor)
df_norm["Taxa Erros (%)"] = normalize(df["Taxa Erros (%)"], invert=True) * pesos["Taxa Erros (%)"]

# Normalizar "Menor Duração" (quanto menor, melhor)
df_norm["Menor Duração"] = normalize(df["Menor Duração"], invert=True) * pesos["Menor Duração"]

# Normalizar "Maior Duração" (quanto menor, melhor)
df_norm["Maior Duração"] = normalize(df["Maior Duração"], invert=True) * pesos["Maior Duração"]

# Normalizar "Duração Média" (quanto menor, melhor)
df_norm["Duração Média"] = normalize(df["Duração Média"], invert=True) * pesos["Duração Média"]

# Normalizar "Conexões Ativas" (quanto menor, melhor)
df_norm["Conexões Ativas"] = normalize(df["Conexões Ativas"], invert=True) * pesos["Conexões Ativas"]

# Normalizar "Tam. Arq. Temp. (GB)" (quanto menor, melhor)
df_norm["Tam. Arq. Temp. (GB)"] = normalize(df["Tam. Arq. Temp. (GB)"], invert=True) * pesos["Tam. Arq. Temp. (GB)"]

# Normalizar "Cache Hit (%)" (quanto maior, melhor)
df_norm["Cache Hit (%)"] = normalize(df["Cache Hit (%)"]) * pesos["Cache Hit (%)"]

# Normalizar "Uso Máx. CPU" (quanto menor, melhor)
df_norm["Uso Máx. CPU"] = normalize(df["Uso Máx. CPU"], invert=True) * pesos["Uso Máx. CPU"]

# Normalizar "Uso Máx. Memória (GB)" (quanto menor, melhor)
df_norm["Uso Máx. Memória (GB)"] = normalize(df["Uso Máx. Memória (GB)"], invert=True) * pesos["Uso Máx. Memória (GB)"]

# Calcular o score composto somando as métricas ponderadas
df_norm["Score"] = df_norm.sum(axis=1) * -1

# Adicionar coluna de Experimento para facilitar o agrupamento (se houver mais de um por experimento)
df_norm["Experimento"] = df["Experimento"]

# Se houver múltiplas linhas por experimento, podemos tirar a média do score
df_result = df_norm.groupby("Experimento")["Score"].mean().reset_index()

# Ordenar do melhor para o pior (quanto maior o score, melhor o desempenho)
df_result = df_result.sort_values(by="Score", ascending=False)

print("Ranking dos Experimentos (melhor desempenho):")
print(df_result)


Ranking dos Experimentos (melhor desempenho):
  Experimento     Score
3      Exp_03  5.296965
4      Exp_04  5.240595
1      Exp_01  5.152517
2      Exp_02  5.099420
0      Exp_00  4.280034


## Segunda Abordagem

Você deve mudar os pesos de acordo com o seu conhecimento na área

In [2]:
import pandas as pd
import numpy as np

# Carrega o DataFrame
df = pd.read_csv('Experimentos.csv')

# Normaliza a coluna removendo o %
df["Uso Máx. CPU"] = df["Uso Máx. CPU"].astype(str).str.replace("%", "")

# Convertendo colunas numéricas (e substituir vírgula decimal por ponto)
numeric_columns = [
    "Nº Usuários", "Taxa Erros (%)", "Menor Duração", "Maior Duração", "Duração Média", "Conexões Ativas",
    "Tam. Arq. Temp. (GB)", "Cache Hit (%)", "Uso Máx. CPU", "Uso Máx. Memória (GB)"
]
for col in numeric_columns:
    df[col] = df[col].astype(str).str.replace(",", ".").astype(float)
# Exemplo de colunas utilizadas:
# "Experimento", "Nº Usuários", "Taxa Erros (%)", "Menor Duração", "Maior Duração", "Cache Hit (%)"

# Vamos definir os critérios a usar e se cada um é de benefício (True) ou de custo (False)
criteria_info = {
    "Nº Usuários": True,                 # Quanto maior número de usuários, melhor
    "Taxa Erros (%)": False,             # Quanto menor a taxa de erros, melhor
    "Menor Duração": False,              # Quanto menor a duração média, melhor
    "Maior Duração": False,              # Quanto menor a duração máxima é melhor
    "Duração Média": False,              # Quanto menor a duração média é melhor
    "Conexões Ativas": False,            # Quanto menos conexoes ativas é melhor
    "Tam. Arq. Temp. (GB)": False,       # Quanto menos arquivo temporario é melhor
    "Cache Hit (%)": True,               # Quanto maior uso da cache é melhor
    "Uso Máx. CPU": False,               # Quanto menos uso de CPU é melhor
    "Uso Máx. Memória (GB)": False       # Quanto menos uso de memória é melhor
}

# Definir os pesos para cada critério (importância relativa)
# Ajuste os valores conforme sua análise fundamentada
weights = {
    "Nº Usuários": 1.0,                 # Quanto maior número de usuários, melhor
    "Taxa Erros (%)": -0.5,             # Quanto menor a taxa de erros, melhor
    "Menor Duração": -0.5,              # Quanto menor a duração média, melhor
    "Maior Duração": -0.5,              # Quanto menor a duração máxima é melhor
    "Duração Média": -1.0,              # Quanto menor a duração média é melhor
    "Conexões Ativas": -0.5,            # Quanto menos conexoes ativas é melhor
    "Tam. Arq. Temp. (GB)": -0.5,       # Quanto menos arquivo temporario é melhor
    "Cache Hit (%)": 0.5,               # Quanto maior uso da cache é melhor
    "Uso Máx. CPU": -0.5,               # Quanto menos uso de CPU é melhor
    "Uso Máx. Memória (GB)": -0.5       # Quanto menos uso de memória é melhor
}

# Normalizando os pesos para que a soma seja 1
total_weight = sum(weights.values())
for crit in weights:
    weights[crit] /= total_weight

# Seleciona apenas as colunas de critérios
criteria_cols = list(criteria_info.keys())
decision_matrix = df[criteria_cols].astype(float).copy()

# 1. Normalização da matriz de decisão (normalização Euclidiana)
norm_decision = decision_matrix.copy()
for crit in criteria_cols:
    norm = np.sqrt((decision_matrix[crit]**2).sum())
    norm_decision[crit] = decision_matrix[crit] / norm

# 2. Multiplicar pelos pesos
weighted_matrix = norm_decision.copy()
for crit in criteria_cols:
    weighted_matrix[crit] = weighted_matrix[crit] * weights[crit]

# 3. Determinar as soluções ideais positiva e negativa
ideal_positive = {}
ideal_negative = {}
for crit in criteria_cols:
    if criteria_info[crit]:  # Critério de benefício
        ideal_positive[crit] = weighted_matrix[crit].max()
        ideal_negative[crit] = weighted_matrix[crit].min()
    else:  # Critério de custo
        ideal_positive[crit] = weighted_matrix[crit].min()
        ideal_negative[crit] = weighted_matrix[crit].max()

# 4. Calcular as distâncias Euclidianas para cada alternativa
def euclidean_distance(row, ideal):
    return np.sqrt(np.sum((row - ideal) ** 2))

# Converter soluções ideais para Series para facilitar o cálculo
ideal_positive_series = pd.Series(ideal_positive)
ideal_negative_series = pd.Series(ideal_negative)

# Inicializa listas para armazenar as distâncias
dist_pos = []
dist_neg = []

for index, row in weighted_matrix.iterrows():
    d_pos = euclidean_distance(row, ideal_positive_series)
    d_neg = euclidean_distance(row, ideal_negative_series)
    dist_pos.append(d_pos)
    dist_neg.append(d_neg)

# 5. Calcular o coeficiente de similaridade (TOPSIS score)
topsis_score = []
for d_pos, d_neg in zip(dist_pos, dist_neg):
    score = d_neg / (d_pos + d_neg)
    topsis_score.append(score)

# 6. Criar um DataFrame com os resultados e ordenar
df_topsis = pd.DataFrame({
    "Experimento": df["Experimento"],
    "TOPSIS Score": topsis_score
})
df_topsis = df_topsis.sort_values(by="TOPSIS Score", ascending=False).reset_index(drop=True)

df_topsis_unique = df_topsis.groupby("Experimento", as_index=False)["TOPSIS Score"].median()
df_topsis_unique = df_topsis_unique.sort_values(by="TOPSIS Score", ascending=False).reset_index(drop=True)

print("Ranking dos Experimentos (por mediana):")
print(df_topsis_unique)


Ranking dos Experimentos (por mediana):
  Experimento  TOPSIS Score
0      Exp_03      0.827700
1      Exp_04      0.826417
2      Exp_02      0.802033
3      Exp_01      0.801455
4      Exp_00      0.647560


## Conclusão

comparar as duas abordagens comparando os resultados , é bom chamar atenção que o topsis é um método já consolidado.

Dizer que o primeiro método foi criado com base na sua experiência e feeling da área e que o segundo foi usado para comparar com uma abordagem já consolidada.

é bom destacar que nos dois casos os experimentos 4 e 3 ficaram próximas em nos primeiros lugares. 